In [ ]:
import os
import json
from tqdm import tqdm

os.chdir('dataset/Diff_Quality_Estimation')


In [ ]:
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from baseform import is_base_form_name, lemma
import json
from contextlib import contextmanager

@contextmanager
def files_dict(files):
    try:
        yield files
    finally:
        for _, file in files.items():
            try:
                file.close()
            except:
                pass


def filter_having_base(file_in, words):
    lemmatizer = WordNetLemmatizer()
    bases = [lemma(lemmatizer, word) for word in words]
    i, j = 0, {base: dict() for base in bases}
    with files_dict({base: open(f'classes/{base}.jsonl', 'w') for base in bases}) as files:
        with open(file_in, 'r') as fr:
            for line in tqdm(fr):
                i += 1
                data = json.loads(line)
                lang = data['lang'] or 'unk'
                words = word_tokenize(line)
                for base in bases:
                    if any(is_base_form_name(lemmatizer, word, base) for word in words):
                        if not lang in j[base]:
                            j[base][lang] = 0
                        j[base][lang] += 1
                        files[base].write(f"{i}\n")
                        #files[base].write(f"[{i},{line.rstrip()}]\n")
    return i, j

In [ ]:
from smells import smells

msg_file = f"comments.jsonl"
filter_having_base(msg_file, smells)